# 17장 에러 처리와 디버깅

#### 이 장에서 다루는 내용
- 브라우저의 에러 보고에 대한 이해
- 에러 처리 
- 자바스크립트 코드 디버그


## 17.1 브라우저 에러 보고
해당 장은 각 브라우저별 에러 보고를 설명이므로 생략


## 17.2 에러 처리
브라우저에서의 에러 처리는 웹 어플레이케이션에서 매우 중요한데도 천천히 도입되었다.   
중요한 점은 웹을 이용하는 사람들이 모두 전문가가 아니라는걸 이해하는 것이다.   
각 브라우저는 자바스크립트 에러에 대해 조금씩 다르게 대처한다.   
따라서 에러를 잘 처리하여 사용자는 무슨 일이 일어난 건지 이해시켜 불안을 해소시켜야 한다.   
이를 위해서는 다양한 자바스크립트 에러를 잡아내서 처리하는 다양한 방법을 반드시 이해해야 한다.   


### 17.2.1 try-catch 문  
ECMA-262 3판에서 자바스크립트 예외를 처리하는 try-catch 문을 도입했다.  

In [ ]:
try {
    // 에러가 생길 수 있는 코드
    window.someNonexistentFunction();
} catch (error) {
    // 에러가 생겼을때 할 일
    console.log('An error happend!');
    console.log('Is error Message > ', error.message);
}

#### finally 절
try-catch 문에는 옵션으로 finally 절을 쓸 수 있는데 이 절의 코드는 항상 실행된다.  
try나 catch 절에서 finally의 실행을 막을 방법은 없으며 return 문이 들어있어도 실행된다. 

In [ ]:
function testFinally() {
    try {
        return 2;
    } catch (error) {
        return 1;
    } finally {
        return 0;
    }
}

testFinally()

#### 에러 타입
코드 실행 중에 발생하는 에러에는 여러 가지 타입이 있다.  
각 에러 타입에는 대응하는 에러 객체 타입이 존재한다.  
ECMA-262에서는 다음 일곱 가지 에러 타입을 정의했다.  

**EvalError**  
전역 함수 eval()에서 발생하는 오류의 인스턴스를 생성합니다.  

**InternalError**  
JavaScript 엔진의 내부에서 오류가 발생했음을 나타내는 오류 인스턴스를 생성합니다.  

**RangeError**  
숫자 변수나 매개변수가 유효한 범위를 벗어났음을 나타내는 오류 인스턴스를 생성합니다.  

**ReferenceError**  
잘못된 참조를 했음을 나타내는 오류 인스턴스를 생성합니다.  

**SyntaxError**  
eval()이 코드를 분석하는 중 잘못된 구문을 만났음을 나타내는 오류 인스턴스를 생성합니다.  

**TypeError**  
변수나 매개변수가 유효한 자료형이 아님을 나타내는 오류 인스턴스를 생성합니다.  

**URIError**  
encodeURI()나 decodeURl() 함수에 부적절한 매개변수를 제공했을 때 발생하는 오류의 인스턴스를 생성합니다.  

In [ ]:
// 특정 오류 처리하기
try {
//     foo.bar();
//     var item = new Array(-20);
} catch (e) {
    if (e instanceof EvalError) {
        console.log('EvalError!');
    } else if (e instanceof RangeError) {
        console.log('RangeError!');
    } else if (e instanceof ReferenceError) {
        console.log('ReferenceError!');
    }
}

### 17.2.2 에러 반환
try-catch 문의 동반자 격인 throw 연산자는 언제든 커스텀 에러를 반환할 수 있다.  
throw 연산자에는 반드시 값이 필요하지만 값의 타입에는 제한이 없다.  
다음 코드는 모두 유효하다.  

`
throw 12345;
throw 'hello world!';
throw true;
throw { name: 'javascirpt' };
`

throw 연산자를 사용하면 코드 실행은 즉시 멈추며 try-catch 문에서 해당 값을 받았을 때에만 계속 진행한다.  

In [ ]:
// 일반적인 오류 던지기 
try {
  throw new Error("이런!");
} catch (e) {
  console.log(e.name + ": " + e.message);
}

In [ ]:
// 커스텀 오류 던지기
function customError(message) {
    this.name = 'customError';
    this.message = message;
}

customError.prototype = new Error();

try {
  throw new customError('Is custom!');
} catch (e) {
  console.log(e.name + ": " + e.message);
}

#### 커스텀 에러를 써야 할 때
커스텀 에러를 쓰면 함수 실행이 왜 실패했는지 매우 중요한 단서를 제공할 수 있다.  
각 브라우저별로 다른 에러 메세지를 던지고 각 메세지만으로 어떻게 문제를 해결할 수 있는지 알기가 힘들다.  
따라서 커스텀 에러를 통해 왜 오류가 일어났는지에 대한 명확한 이유를 제공할 수 있다.  

In [ ]:
function process(values) {
    console.log(values)
    if (!(values instanceof Array)) {
        throw new Error('process(): Argument must be an array.');
    }
    
    values.sort();
    
    for (var i = 0, len = values.length; i < len; i++) {
        if (values[i] > 100) {
            return values[i];
        }
    }
    
    return -1;
}

process(100);

#### 커스텀 에러 vs try-catch
try-catch 문의 목적은 브라우저가 에러를 기본 방식대로 처리하지 못하게 막는 것이고  
커스텀 에러의 목적은 에러가 일어난 이유를 제공하는 것이다.  

### 17.2.3 error 이벤트 
try-catch 문으로 처리하지 않은 에러가 발생하면 window 객체에서 error 이벤트가 발생한다.  
이 이벤트는 웹 브라우저들이 처음부터 지원하던 것이라서 IE, 파이어폭스, 크롬의 하위 호환성을 위해 형식을 유지하고 있다. (오페라와 사파리는 지원안함)  
onerror 이벤트 핸들러는 어떤 브라우저에서도 event 객체를 생성하지 않으며 에러 메세지와 URL, 행 번호 세가지를 매개변수로 받는다.  

In [ ]:
// 노드 환경에서는 window 객체가 없으므로 브라우저에서 테스트
window.onerror = function (message, url, line) {
    console.log(message);
}

### 17.2.4 에러 처리 전략
전통적으로 웹 어플리케이션의 에러 처리는 서버 몫이었다.  
에러 처리의 요점은 에러 패턴을 분석하여 에러의 근본 원인을 추적하고 에러가 얼마나 많은 사용자에게 영향을 미치는지 이해하자는 것이다. 
웹 어플리케이션의 자바스크립트 레벨에서도 에러 처리 전략을 세워야 하며 그 중요성은 서버 레벨과 다르지 않다.  
개발자라면 코드가 언제, 어떻게 실패하는지 잘 이해해야 하며 문제를 추적할 수 있는 시스템을 갖춰야 한다.   


### 17.2.5 에러가 생길 수 있는 곳 찾기  
에러 처리에서 가장 중요한 일은 먼저 코드에서 에러가 생길 수 있는 곳을 찾는 일이다.  
자바스크립트는 데이터 타입이 느슨하며 함수의 매개변수를 검사하지 않으므로 코드를 실행해야만 에러를 확인할 수 있을 때가 많다.  
일반적으로 세가지 범주의 에러에 주목해야 한다.  
- 타입 강제 에러
- 데이터 타입 에러 
- 통신 에러 


**타입 강제 에러**
값의 데이터 타입을 자동으로 바꾸는 연산자나 기타 언어 구조를 사용했을 때 발생한다.  
가장 흔하게 말생하는 경우는 두가지이다. 
- ==와 != 연산자 (서로 다른 타입을 비교하기 전에 자동으로 타입을 바꿈)
- if와 for, while 같은 흐름 제어문에 불리언이 아닌 값을 쓴 경우

In [ ]:
// 연산자 예시 
console.log(5 == '5');
console.log(5 === '5');
console.log(1 == true);
console.log(1 === true);

In [ ]:
// 흐름 제어문 예시 
function concat(str1, str2, str3) {
    var result = str1 + str2;
    if (str3) {  // 이렇게 하지 마세요!
        result += str3/
    }
    
//     if (typeof str3 == 'string') {  // 올바른 비교
//         result += str3;
//     }
        
    return result
}

**데이터 타입 에러**
자바스크립트는 느슨한 타입을 사용하므로 변수나 함수 배개변수에 정확한 타입의 데이터가 사용됐는지 미리 비교하지 않는다.  

In [ ]:
// 타입 체크
function getQueryString(url){
    if (typeof url == 'string') {
        var pos = url.indexOf('?');
        if (pos > -1) {
            return url.substring(pos + 1);
        }
    }
    
    return '';
}

In [ ]:
// 타입 체크
function reverseSort(values) {
    if (values instanceof Array) {
        values.sort();
        values.reverse();
    }
}

**통신 에러**


### 17.2.6 심각한 에러와 그렇지 않은 에러 구별  

**심각하지 않은 에러**
- 사용자의 주요 작업을 방해하지 않는 에러 
- 페이지의 일부분에만 영향을 끼치는 에러 
- 복구 가능한 에러 
- 다시 시도해보면 일어나지 않을 수 있는 에러  

**심각한 에러** 
- 애플리케이션 진행을 완전히 막는 에러 
- 사용자의 주요 목적을 뚜렷이 방해하는 에러 
- 다른 에러를 일으키는 에러 

### 17.2.7 서버에 에러 기록 
해당 내용은 서버에 에러 기록하는것을 추천한다는 설명글이라 생략

## 17.3 디버그 테크닉  

**3가지 방법을 활용하기**
- 콘솔에 기록
- 페이지에 메시지 기록
- 커스텀 에러 

**콘솔 객체**  
https://developer.mozilla.org/ko/docs/Web/API/Console

**크롬 개발자 도구 활용하기**  
https://subicura.com/2018/02/14/javascript-debugging.html

## 17.5 요약 
현대의 복잡한 웹 애플리케이션에서는 자바스크립트 에러 처리가 대단히 중요하다.  
어디에서 에러가 발생할지, 어떻게 해결할지 예측하지 못한다면 사용자 경험을 저해하고 사용자를 화나게 만들 수도 있다.  
대부분의 브라우저는 기본적으로 자바스크립트 에러를 사용자에게 보여주지 않으므로 디버그할 때는 에러 표시를 켜야 한다.  
하지만 배포 단계에서는 에러 표시를 모두 꺼야 한다.  
다음은 브라우저가 자바스크립트 에러를 관리하지 않게 막는 방법이다.  
- try-catch 문은 에러가 발생할 수도 있는 상황에 적합하며 브라우저가 에러를 처리하게 내버려두지 않고 적절히 처리할 수 있다.  
- window.onerror 이벤트 핸들러를 쓸 수도 있다. 이 이벤트는 try-catch로 처리하지 않은 에러를 모두 받는다. 


**참고하기 좋은 자료**
- https://rinae.dev/posts/how-to-handle-errors-1
- https://github.com/FEDevelopers/tech.description/wiki/%EC%9E%90%EB%B0%94%EC%8A%A4%ED%81%AC%EB%A6%BD%ED%8A%B8-%EC%97%90%EB%9F%AC-%ED%95%B8%EB%93%A4%EB%A7%81-:-%EC%8B%A0%EB%A2%B0-%ED%95%A0-%EB%A7%8C%ED%95%9C-%EA%B0%80%EC%9D%B4%EB%93%9C
